In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\ENGRACED\\Documents\\Project\\datascience\\farm_copilot\\training\\farm_copilot_model_trainer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBasedModelConfig:
    root_dir: Path
    local_data_file: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: tuple
    params_learning_rate: float
    params_include_top: bool
    params_weights: str  
    params_offset: int
    

In [6]:
from farm_copilot.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from farm_copilot.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,          
        ) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_prepare_base_model_config(self) -> PrepareBasedModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBasedModelConfig(
            root_dir=Path(config.root_dir),
            local_data_file=Path(self.config.data_ingestion.local_data_file),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_offset=self.params.OFFSET,
        )

        return prepare_base_model_config




In [8]:
import os
import keras

In [13]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBasedModelConfig) -> None:
        self.config = config


    def get_base_model(self):
        self.model = keras.applications.Xception(
            input_shape=tuple(self.config.params_image_size),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(model, classes: int, freeze_all: bool, freeze_till: int, learning_rate: float, offset: int):
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        scaled_layer = keras.layers.Rescaling(scale=1./127.5, offset=offset)(model.input)
        avg_pool = keras.layers.GlobalAveragePooling2D()(scaled_layer)
        dropout = keras.layers.Dropout(0.2)(avg_pool)
        output = keras.layers.Dense(units=classes, activation='softmax')(dropout)
        
        full_model = keras.Model(inputs=model.input,outputs= output)
        
        full_model.compile(
            optimizer=keras.optimizers.SGD(learning_rate=learning_rate),
            loss=keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=len(os.listdir(Path(self.config.local_data_file))),
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate, 
            offset=self.config.params_offset
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)


    @staticmethod
    def save_model(path: Path, model: keras.Model):
        keras.saving.save_model(model=model, filepath=path)
        
  

In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-03-26 15:52:20,399: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-26 15:52:20,402: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-26 15:52:20,404: INFO: common: created directory at: artifacts]
[2024-03-26 15:52:20,405: INFO: common: created directory at: artifacts/prepare_base_model]


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 3)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 62)             │           248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 248 (992.00 B)

 Trainable params: 248 (992.00 B)

 Non-trainable params: 0 (0.00 B)